In [1]:
!pip install kagglehub


import os
import kagglehub
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


/Users/ananthakrishnan/Documents/Code/ML/Mini/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
datasetpath = kagglehub.dataset_download('chethuhn/network-intrusion-dataset')
print(f"Dataset downloaded to: {datasetpath}")

Dataset downloaded to: /Users/ananthakrishnan/.cache/kagglehub/datasets/chethuhn/network-intrusion-dataset/versions/1


In [3]:
datasetfile = os.path.join(datasetpath, 'network_intrusion_data.csv')

In [4]:
files = os.listdir(datasetpath)
csv_files = []
for file in files:
    if file.endswith('.csv'):
        csv_files.append(file)

In [5]:
data = pd.DataFrame()

In [6]:
print(data.head())

Empty DataFrame
Columns: []
Index: []


In [7]:
for file in csv_files:
        file_path = os.path.join(datasetpath, file)
        temp_data = pd.read_csv(file_path)
        data = pd.concat([data, temp_data], ignore_index=True)

In [8]:
col_names = {col: col.strip() for col in data.columns}
data.rename(columns = col_names, inplace = True)

In [9]:
data.replace([np.inf,-np.inf],np.nan,inplace=True)
data.dropna(inplace=True)
data.drop_duplicates(inplace=True)

In [10]:
target = data['Label']
features = data.drop(columns=['Label'])

In [11]:
categorical_columns = features.select_dtypes(include=['object']).columns.tolist()
encoder = LabelEncoder()
for col in categorical_columns:
    features[col] = encoder.fit_transform(features[col].astype(str))

In [12]:
numerical_columns = features.select_dtypes(include=['float64', 'int64']).columns.tolist()
scaler = StandardScaler()
features[numerical_columns] = scaler.fit_transform(features[numerical_columns])

In [13]:
n_features = features.shape[1]
pca = PCA(n_components=min(n_features, 50)) 
features_pca = pca.fit_transform(features)

In [15]:
label_encoder = LabelEncoder()
target_encoded = label_encoder.fit_transform(target)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(features_pca, target, test_size=0.2, random_state=42)
model = xgb.XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss')
model.fit(X_train, y_train)

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14], got ['BENIGN' 'Bot' 'DDoS' 'DoS GoldenEye' 'DoS Hulk' 'DoS Slowhttptest'
 'DoS slowloris' 'FTP-Patator' 'Heartbleed' 'Infiltration' 'PortScan'
 'SSH-Patator' 'Web Attack � Brute Force' 'Web Attack � Sql Injection'
 'Web Attack � XSS']

In [ ]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")